<td>
<a href="https://colab.research.google.com/github/raoulg/MADS-DAV/blob/main/notebooks/6.3.3-tanach_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</td>

In [ ]:
import pandas as pd
from pathlib import Path

datadir = Path("../data/processed")
datafile = datadir / Path("tanach.parquet")
df = pd.read_parquet(datafile)
df.head()

We start with loading the preprocessed text.

In [ ]:
result = df.groupby("book")['text'].apply(lambda x: ''.join(x)).str.len().reset_index(name='char_count')
result

Let's chunk all text into chunks of 5000 characters.

In [ ]:
def split_into_chunks(text, chunk_size=5000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size) if len(text[i:i+chunk_size]) == chunk_size]

# Group by "book", join "text", then split into chunks
# another approach (maybe better?) could be to split on spaces and then chunk into k words instead of characters
result = df.groupby("book")['text'].apply(lambda x: ''.join(x)).apply(lambda x: split_into_chunks(x)).reset_index(name='chunks')
result_exploded = result.explode('chunks').reset_index(drop=True)
result_exploded


In [ ]:
result_exploded.dropna(axis=0, inplace=True)

We will take a subset of the texts.

In [ ]:
books = ["Genesis", "Exodus", "Leviticus", "Numbers", "Deuteronomy", "Joshua"]

text = result_exploded[result_exploded['book'].isin(books)]
text


And now we can apply our model to the text.

In [ ]:
from wa_analyzer.model import TextClustering
clustering = TextClustering()
labels = text['book']

X = clustering(text['chunks'], k=0, batch=False, method="PCA", labels=labels)
